# First bar & pivot points & Xgboost => day Low - Predict

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import math

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.constrained_layout.use'] = True
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
def bar9h(tick):
    tick=tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]

data['Close_9h'] = data['Close']
day_data = data.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h
    })
day_data.dropna(subset=['Close'], inplace=True)

In [5]:
day_data['Pivot'] = day_data.apply(lambda row: (row['High'] + row['Low'] + row['Close']) / 3, axis=1)
day_data['Pivot'] = day_data['Pivot'].shift(1)
day_data['first_open_vs_pivot'] = day_data.apply(lambda r: 1000 * (r['Open'] - r['Pivot']) / r['Open'], axis=1)
day_data['first_close_vs_pivot'] = day_data.apply(lambda r: 1000 * (r['Close_9h'] - r['Pivot']) / r['Close_9h'], axis=1)
day_data['High_s'] = day_data['High'].shift(1)
day_data['Low_s'] = day_data['Low'].shift(1)

In [6]:
day_data = day_data[(day_data.first_close_vs_pivot > -20) & (day_data.first_close_vs_pivot < 20) & (day_data.first_open_vs_pivot > -20) & (day_data.first_open_vs_pivot < 20)]

In [7]:
day_data

,Open,Close,High,Low,Close_9h,Pivot,first_open_vs_pivot,first_close_vs_pivot,High_s,Low_s
Date,,,,,,,,,,
2018-08-14,955.5,959.3,961.8,952.1,954.9,950.366667,5.372405,4.747443,954.6,942.3
2018-08-15,958.0,946.6,962.3,946.6,958.7,957.733333,0.278358,1.008310,961.8,952.1
2018-08-16,942.4,947.0,947.3,935.5,942.0,951.833333,-10.009904,-10.438783,962.3,946.6
2018-08-17,952.2,947.0,954.6,945.0,953.0,943.266667,9.381783,10.213361,947.3,935.5
2018-08-20,946.1,945.0,951.5,942.6,945.8,948.866667,-2.924286,-3.242405,954.6,945.0
...,...,...,...,...,...,...,...,...,...,...
2024-08-08,1248.4,1240.1,1251.3,1237.4,1246.4,1250.400000,-1.602051,-3.209243,1255.5,1245.7
2024-08-09,1249.1,1264.9,1266.2,1247.7,1248.5,1242.933333,4.936888,4.458684,1251.3,1237.4
2024-08-12,1262.5,1270.0,1270.4,1258.7,1262.9,1259.600000,2.297030,2.613033,1266.2,1247.7


In [8]:
data_test = day_data.copy()
data_test = data_test[data_test.index > '2024-01-01 00:00:00']
data_train = day_data[day_data.index < '2024-01-01 00:00:00']

In [9]:
import xgboost as xg 
from sklearn.metrics import mean_squared_error as MSE

In [10]:
feature_cols = ["Low_s", "first_open_vs_pivot", "first_close_vs_pivot", "Pivot"]
# Splitting 
X_train = data_train[feature_cols]
y_train = data_train[['Low']]
X_test = data_test[feature_cols]
y_test = data_test[['Low']]
  
# Instantiation 
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 100, seed = 123)
  
# Fitting the model 
xgb_r.fit(X_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
# Predict the model 
y_pred = xgb_r.predict(X_test) 
  
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, y_pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  10.721172


In [12]:
X_result = X_test.copy()
X_result = X_result.assign(Predicts=y_pred)
X_result = X_result[['Predicts']]
data_test = pd.merge(data_test, X_result, left_index=True, right_index=True, how="left")
data_test.dropna(inplace=True)

In [13]:
data_test

,Open,Close,High,Low,Close_9h,Pivot,first_open_vs_pivot,first_close_vs_pivot,High_s,Low_s,Predicts
Date,,,,,,,,,,,
2024-01-02,1138.5,1133.5,1141.8,1131.0,1138.9,1135.000000,3.074220,3.424357,1139.5,1130.9,1128.232300
2024-01-03,1130.5,1148.3,1148.3,1126.5,1128.3,1135.433333,-4.363851,-6.322196,1141.8,1131.0,1101.737793
2024-01-04,1146.0,1156.5,1171.0,1145.5,1146.1,1141.033333,4.333915,4.420789,1148.3,1126.5,1139.200073
2024-01-05,1156.1,1166.0,1166.0,1154.2,1158.8,1157.666667,-1.355131,0.978023,1171.0,1145.5,1144.824829
2024-01-08,1166.1,1162.0,1173.3,1160.6,1167.8,1162.066667,3.458823,4.909516,1166.0,1154.2,1160.839111
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-08,1248.4,1240.1,1251.3,1237.4,1246.4,1250.400000,-1.602051,-3.209243,1255.5,1245.7,1214.971924
2024-08-09,1249.1,1264.9,1266.2,1247.7,1248.5,1242.933333,4.936888,4.458684,1251.3,1237.4,1234.726440
2024-08-12,1262.5,1270.0,1270.4,1258.7,1262.9,1259.600000,2.297030,2.613033,1266.2,1247.7,1251.222290
